# Mnist classification with NNs
A first example of a simple Neural Network, applied to a well known dataset.from tensorflow.keras.layers import Input, Dense
tensorflow.keras.layers TensorFlow 的 Keras API 提供的神经网络层模块。
Input 用于定义神经网络的输入层，指定输入数据的形状。
Dense 全连接密集层，是神经网络中常用的一种层，所有神经元相互连接。

In [8]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras import utils
import numpy as np

Let us load the mnist dataset

tensorflow.keras.datasets：Keras 提供的一些常见数据集。
mnist：手写数字识别数据集（28x28 像素的灰度图片，0-9 这 10 个类别）。

x_train：训练集的图像数据，形状 (60000, 28, 28)，代表 60,000 张 28×28 的灰度图片。
y_train：训练集的标签数据，形状 (60000,)，代表 60,000 个标签（每个是 0-9 之间的整数，表示手写数字的类别）。
x_test：测试集的图像数据，形状 (10000, 28, 28)，代表 10,000 张 28×28 的灰度图片。
y_test：测试集的标签数据，形状 (10000,)，代表 10,000 个标签（0-9 之间的整数）。

x_train 和 x_test 是 3D NumPy 数组，格式：(样本数量, 高度, 宽度)。
y_train 和 y_test 是 1D NumPy 数组，格式：(样本数量,)，每个值是 0-9 之间的整数，表示数字类别

In [18]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [19]:
print(x_train.shape)
print("pixel range is [{},{}]".format(np.min(x_train),np.max(x_train)))

(60000, 28, 28)
pixel range is [0,255]


[link text](https://)We normalize the input in the range [0,1]

将像素值归一化到 [0, 1] 之间，提高模型训练效果。
MNIST 数据集的像素值范围是 0-255，即 灰度图像的像素值范围。
astype('float32') 转换数据类型，避免整数运算导致的精度问题。
为什么要这么做？
深度学习模型对数值范围较敏感：
像素值如果保持 0-255，可能导致较大的梯度变化，影响模型训练的稳定性。
归一化到 [0, 1]，能让模型更容易学习。
避免溢出问题：
深度学习中的计算涉及大量的矩阵运算，如果不归一化，容易导致梯度爆炸或消失。
例如，计算 exp(x)（如 softmax 层）时，过大的输入可能导致数值溢出。

将 28×28 的二维图像数据转换成 1D 向量（784** 维度）**，方便输入 Dense 层。
为什么要这么做？
全连接层 (Dense) 只能接受 1D 输入：
Dense 层输入的数据需要是一维向量，而 x_train 的原始形状是 (60000, 28, 28)（3D）。
reshape(60000, 28*28) 变成 (60000, 784)，每张图片变成长度为 784 的向量，适合 Dense 层。

In [20]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train = np.reshape(x_train,(60000,28*28))
x_test = np.reshape(x_test,(10000,28*28))

\The output of the network will be a proability distribution over the different categories. Similarly, we generate a ground truth distribution, and the training objective will consist in minimizing their distance (categorical crossentropy). The ground truth distribution is the so called "categorical" distribution: if x has label l, the corresponding categorical distribution has probaility 1 for the category l, and 0 for all the others.

适配神经网络的输出层（Softmax）
使用交叉熵损失函数（Categorical Crossentropy）
避免整数标签影响学习效果
提高计算效率

问题 1：神经网络会误解整数之间的“大小关系”
如果 y_train 直接用整数表示（如 0-9），模型可能会误解类别之间的关系。

问题 2：Softmax 输出层需要 One-Hot 形式
通常，分类模型的最后一层是 softmax 层：

问题 3：交叉熵损失函数需要 One-Hot
在分类任务中，我们通常使用交叉熵损失函数（categorical_crossentropy）：

 什么时候可以不用 One-Hot？
如果你的输出层使用 sparse_categorical_crossentropy，你不需要 One-Hot 编码：

这种损失函数适用于整数标签（y_train 仍然是 0-9）。
适用于大类别分类任务（如 1000 个类别的 ImageNet）。
但在普通分类任务（如 MNIST）中，One-Hot 编码更常用。



In [21]:
print(y_train[0])
y_train_cat = utils.to_categorical(y_train)
print(y_train_cat[0])
y_test_cat = utils.to_categorical(y_test)

5
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


Our first Netwok just implements logistic regression

Input(shape=(784,)) 定义输入张量（Tensor）。
Dense(64, activation='relu')(input_layer) 创建权重 W 和偏置 b，并应用 ReLU 激活函数。
Model(inputs, outputs) 组合这些层，形成完整的计算图。

Input() 定义输入层，用于指定输入数据的形状。
shape=(28*28,) 表示输入是一个 784 维的一维向量。
为什么是 (28*28,)？
MNIST 数据集中的图片是 28×28 的灰度图像。
在 x_train = np.reshape(x_train, (60000, 28*28)) 这行代码中，我们将 28×28 的图片展平成 784 维的向量，所以 Input 层也要匹配这个形状


Dense(10, activation='softmax') 定义了一个 全连接层（Dense 层），并使用 softmax 作为激活函数。
参数解释
10：表示该层的 神经元个数，即输出是一个 10 维向量。
activation='softmax'：
softmax 计算 10 个类别的概率分布，保证所有类别的概率加起来等于 1。
适用于 多分类任务，如 MNIST（0~9，共 10 个类别）。
计算过程
如果 xin 传入的是一个 (60000, 784) 的数据：

Dense(10) 作用：
计算 权重矩阵 (W) 和 偏置 (b)：

z=XW+b
其中：
X 是输入数据（形状：(batch_size, 784)）。
W 是权重矩阵（形状：(784, 10)）。
b 是偏置（形状：(10,)）。
结果 z 形状是 (batch_size, 10)。

softmax(z)：
将 z 转换为概率分布

In [23]:
xin = Input(shape=(28*28,))
res = Dense(10,activation='softmax')(xin)

mynet = Model(inputs=xin,outputs=res)

In [26]:
mynet.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 10)                  │           7,850 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,850 (30.66 KB)

 Trainable params: 7,850 (30.66 KB)

 Non-trainable params: 0 (0.00 B)

Now we need to compile the network.
In order to do it, we need to pass two mandatory arguments:


*   the **optimizer**, in charge of governing the details of the backpropagation algorithm
*   the **loss function**

Several predefined optimizers exist, and you should just choose your favourite one. A common choice is Adam, implementing an adaptive lerning rate, with momentum

Optionally, we can specify additional metrics, mostly meant for monitoring the training process.
mynet.compile() 用于配置模型的训练方式，定义：

优化器 (optimizer)：决定如何更新权重参数，使损失最小化。
损失函数 (loss)：衡量模型预测值和真实值之间的误差。
评估指标 (metrics)：训练时显示的性能指标（如准确率）。

Adam（Adaptive Moment Estimation）是最常用的深度学习优化器，它结合了：

Momentum（动量）：让优化过程更平滑，防止震荡。
RMSProp（自适应学习率）：自动调整每个参数的学习率，避免参数更新过大或过小。
为什么选择 Adam？
✅ 自动调整学习率，适用于大多数任务
✅ 训练更快，比 SGD（随机梯度下降）收敛更快
✅ 效果稳定，适用于各种深度学习任务（图像分类、NLP 等）

替代方案

optimizer='sgd'（随机梯度下降）：简单，但收敛速度慢。
optimizer='rmsprop'（RMSProp）：适用于递归神经网络（RNN）。
optimizer='adamax'：Adam 的变种，适用于非常稀疏的数据。

In [27]:
mynet.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Finally, we fit the model over the trianing set.

Fitting, just requires two arguments: training data e ground truth, that is x and y. Additionally we can specify epochs, batch_size, and many additional arguments.

In particular, passing validation data allow the training procedure to measure loss and metrics on the validation set at the end of each epoch.

In [28]:
mynet.fit(x_train,y_train_cat, shuffle=True, epochs=10, batch_size=32,validation_data=(x_test,y_test_cat))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8097 - loss: 0.7262 - val_accuracy: 0.9144 - val_loss: 0.3078
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9144 - loss: 0.3066 - val_accuracy: 0.9231 - val_loss: 0.2798
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9181 - loss: 0.2858 - val_accuracy: 0.9240 - val_loss: 0.2713
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9237 - loss: 0.2753 - val_accuracy: 0.9246 - val_loss: 0.2707
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9263 - loss: 0.2618 - val_accuracy: 0.9263 - val_loss: 0.2688
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9295 - loss: 0.2552 - val_accuracy: 0.9273 - val_loss: 0.2634
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9274 - loss: 0.2593 - val_accuracy: 0.9273 - val_loss: 0.2642
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9277 - loss: 0.2614 -

In [36]:
xin = Input(shape=(784,))
x = Dense(100,activation='relu')(xin)
res = Dense(10,activation='softmax')(x)

mynet2 = Model(inputs=xin,outputs=res)

In [37]:
mynet2.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)           │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
mynet2.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [35]:
mynet2.fit(x_train,y_train_cat, shuffle=True, epochs=10, batch_size=32,validation_data=(x_test,y_test_cat))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8791 - loss: 0.4353 - val_accuracy: 0.9619 - val_loss: 0.1323
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9633 - loss: 0.1238 - val_accuracy: 0.9719 - val_loss: 0.0939
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9770 - loss: 0.0793 - val_accuracy: 0.9680 - val_loss: 0.0995
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9813 - loss: 0.0615 - val_accuracy: 0.9763 - val_loss: 0.0769
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9869 - loss: 0.0437 - val_accuracy: 0.9774 - val_loss: 0.0754
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9894 - loss: 0.0338 - val_accuracy: 0.9765 - val_loss: 0.0823
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9924 - loss: 0.0265 - val_accuracy: 0.9772 - val_loss: 0.0758
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9938 - loss: 0.0215 -

An amazing improvement. WOW!

# Exercises

1.   Add additional Dense layers and check the performance of the network
2.   Replace 'relu' with different activation functions
3. Adapt the network to work with the so called sparse_categorical_crossentropy
4. the fit function return a history of training, with temporal sequences for all different metrics. Make a plot.

